In [5]:
import matplotlib

%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import numpy as np

from matplotlib import pyplot as plt
import scipy
from scipy.ndimage import gaussian_filter


import csv

import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

from skimage.segmentation import watershed
from skimage.feature import peak_local_max

import cv2
import os

import scipy.misc
#import skimage.filter

import parmap
import glob

import numpy as np
import matplotlib.pyplot as plt
import scipy.misc

from PIL import Image


Autosaving every 180 seconds


In [2]:

def grab_data_from_directories(root_dir, name, data):

    frames = []
    frame_ids = []
    locs_array = []
    tracks = []

    ctr = 0
    max_frames = 5000
    for k in range(data.shape[0]):
        track = data[k,0]
        frame_start = data[k,1]
        frame_end = data[k,2]
        n_frames = data[k, 3]
        print (track, frame_start ,frame_end, " # frames: ", n_frames , ' count: ', ctr)
        #     
        dir_ = os.path.join(root_dir,str(track))

        fnames = np.sort(glob.glob(dir_+"/*.npz"))


        for n in trange(frame_start, frame_end+1,1):
            res = np.int32([i.find('frame_'+str(n).zfill(7)) for i in fnames])
            idx = np.where(res>=0)[0]
            if idx.shape[0]==0:
                continue
            data2 = np.load(fnames[idx][0])

            frames.append(data2['frame'])
            frame_ids.append(data2['frame_id'])
            locs_array.append(data2['locs'])
            tracks.append(data2['track_id_sleap'])

            ctr+=1

            if ctr>=max_frames:
                break

        if ctr>=max_frames:
            break

    np.savez('/home/cat/'+name+'.npz',
             frames = frames,
             frame_ids = frame_ids,
             locs_array = locs_array,
             tracks = tracks
            )
    
def return_selected(data, start, end):
    
    track_ids = data[:,0]
    #print (track_ids)
    female = np.zeros((track_ids.shape[0],4),'float32')
    female[:,0] = track_ids
    female[:,1:3] = data[:,start:end]
    female[:,3] = data[:,end]
    
    idx = np.where(np.isnan(female))
    female[idx]=0
    female=np.int32(female)
    idx = np.where(female[:,1]==0)[0]
    female = np.delete(female, idx, axis=0)
    
    print ("# of examples: ", np.sum(female[:,3]))
    
    return female
    


In [95]:
##########################################################
############ LOAD TRACKS FROM GOOGLE SPREADSHEET #########
##########################################################
fname = '/home/cat/Downloads/manual_id_grouping_2020_3_15_11_53_51_617746_compressed - Sheet.csv'

from numpy import genfromtxt
data = genfromtxt(fname, delimiter=',',dtype='float32')

# load 2nd and 3rd column for each group of data
female = return_selected(data, 1,3)
male = return_selected(data,4,6)    
pup1 = return_selected(data,7,9)
pup2 = return_selected(data,10,12)

print (female[:2])

# of examples:  10441
# of examples:  5057
# of examples:  5138
# of examples:  5982
[[  827 21204 21487   283]
 [  827 21497 22080   583]]


In [96]:
###################################################
############ LOAD TRACKS FROM SPREADSHEET #########
###################################################
from tqdm import trange
import json

root_dir = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/cnn_output/'

name = 'pup2'
data = pup2
    
grab_data_from_directories(root_dir, name, data)



  0%|          | 0/143 [00:00<?, ?it/s]

749 19744 19784  # frames:  40  count:  0
1521 43178 43320  # frames:  142  count:  40


  7%|▋         | 12/166 [00:00<00:01, 117.11it/s]

1521 47407 47572  # frames:  165  count:  170


  2%|▏         | 8/336 [00:00<00:04, 72.25it/s]

1732 65942 66277  # frames:  335  count:  336


 47%|████▋     | 159/336 [00:01<00:02, 81.85it/s]


KeyboardInterrupt: 

In [3]:
data = np.load('/home/cat/pup2.npz')
frames = data['frames']
print (frames.shape)

(5000, 500, 500)


In [6]:
plt.imshow(frames[0])
plt.show()

In [9]:
data2 = np.load('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/cnn_output/0/frame_0000000_id_0.npz')
frame = data2['frame']
print (frame.shape)
plt.imshow(frame)
plt.show()

(500, 500, 3)
